Data Analyst Take Home Exercise for Colton Radenbaugh

# Set-Up

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as db

# Part I - Explore the Data
-  Initially here I'd like to get a high level view of what is in each of these csv's with an understanding what data type the columns are, unique values, amount of nulls, and if there are duplicate values where I might not expect to find them.

- I'm pulling in the provided csv data files that for the exercie. First through panadas for some initial data exploration and then I'll add them to a SQLite database so I can join and analyze them further.

In [2]:
products_df = pd.read_csv('data_files\\PRODUCTS_TAKEHOME.csv')
transaction_df = pd.read_csv('data_files\\TRANSACTION_TAKEHOME.csv')
user_df = pd.read_csv('data_files\\USER_TAKEHOME.csv')

### Products Data Set

In [3]:
print(products_df.head())
print('----------------                         -------------------------')
print(products_df.info())

          CATEGORY_1              CATEGORY_2                   CATEGORY_3  \
0  Health & Wellness           Sexual Health  Conductivity Gels & Lotions   
1             Snacks           Puffed Snacks         Cheese Curls & Puffs   
2  Health & Wellness               Hair Care        Hair Care Accessories   
3  Health & Wellness               Oral Care                   Toothpaste   
4  Health & Wellness  Medicines & Treatments               Essential Oils   

  CATEGORY_4                                       MANUFACTURER  \
0        NaN                                                NaN   
1        NaN                                                NaN   
2        NaN                           PLACEHOLDER MANUFACTURER   
3        NaN                                  COLGATE-PALMOLIVE   
4        NaN  MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...   

             BRAND       BARCODE  
0              NaN  7.964944e+11  
1              NaN  2.327801e+10  
2          ELECSOP  4.618178e

- This PRODUCTS data set seems pretty straight forward as a dimesions table for the different type of items that we could find with a user's purchases. 
There are few potential issues with nulls that I think could cause some issues around null Barcodes which appears to be the key for joining this table to the Transactions data set after looking at the ERM. Also, products that have null values for any of the other dimesions will cause issues if I try and aggregate results on those dimensions, so I would need to omit those nulls when doing aggregation or bucket them into a catch all "other" category.

In [4]:
products_df.describe(include = "object")

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND
count,845441,844128,784986,67459,619078,619080
unique,27,121,344,127,4354,8122
top,Health & Wellness,Candy,Confection Candy,Lip Balms,PLACEHOLDER MANUFACTURER,REM BRAND
freq,512695,121036,56965,9737,86902,20813


- Could be interesting to explore the vast amout of Health & Wellness products, since they are more than half of the data set. Though I'll have to see how prevalent they are in the transactions. 

In [5]:
sorted(products_df['CATEGORY_1'].astype(str).unique())
# sorted(products_df['CATEGORY_2'].astype(str).unique()) 
# sorted(products_df['CATEGORY_3'].astype(str).unique()) 
# sorted(products_df['CATEGORY_4'].astype(str).unique()) 
# sorted(products_df['MANUFACTURER'].astype(str).unique()) 
# sorted(products_df['BRAND'].astype(str).unique()) 

['Alcohol',
 'Animals & Pet Supplies',
 'Apparel & Accessories',
 'Arts & Entertainment',
 'Baby & Toddler',
 'Beauty',
 'Beverages',
 'Dairy',
 'Deli & Bakery',
 'Electronics',
 'Frozen',
 'Health & Wellness',
 'Home & Garden',
 'Household Supplies',
 'Luggage & Bags',
 'Mature',
 'Meat & Seafood',
 'Media',
 'Needs Review',
 'Office & School',
 'Pantry',
 'Produce',
 'Restaurant',
 'Snacks',
 'Sporting Goods',
 'Toys & Games',
 'Vehicles & Parts',
 'nan']

- After taking a scan of the unique values for all of the labeling columns of a product the data seems clean in the fact that there appears to be no issue with labels seeming to be unique but are in fact duplicates that only differ with the addition of a space or whitespace character. This gives me confidence for any aggregation that may need to be done on these dimensions.

### Users Data Set

In [6]:
print(user_df.head())
print('----------------                         -------------------------')
print(user_df.info())

                         ID               CREATED_DATE  \
0  5ef3b4f17053ab141787697d  2020-06-24 20:17:54.000 Z   
1  5ff220d383fcfc12622b96bc  2021-01-03 19:53:55.000 Z   
2  6477950aa55bb77a0e27ee10  2023-05-31 18:42:18.000 Z   
3  658a306e99b40f103b63ccf8  2023-12-26 01:46:22.000 Z   
4  653cf5d6a225ea102b7ecdc2  2023-10-28 11:51:50.000 Z   

                  BIRTH_DATE STATE LANGUAGE  GENDER  
0  2000-08-11 00:00:00.000 Z    CA   es-419  female  
1  2001-09-24 04:00:00.000 Z    PA       en  female  
2  1994-10-28 00:00:00.000 Z    FL   es-419  female  
3                        NaN    NC       en     NaN  
4  1972-03-19 00:00:00.000 Z    PA       en  female  
----------------                         -------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ID            100000 non-null  object
 1   CREATED_DATE  100

In [7]:
user_df.describe(include= "object")

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
count,100000,100000,96325,95188,69492,94108
unique,100000,99942,54721,52,2,11
top,5ef3b4f17053ab141787697d,2023-01-12 18:30:15.000 Z,1970-01-01 00:00:00.000 Z,TX,en,female
freq,1,2,1272,9028,63403,64240


In [8]:
sorted(user_df['GENDER'].astype(str).unique())

["My gender isn't listed",
 'Non-Binary',
 'Prefer not to say',
 'female',
 'male',
 'nan',
 'non_binary',
 'not_listed',
 'not_specified',
 'prefer_not_to_say',
 'transgender',
 'unknown']

- The dimesion columns for the user data seems clean with no whitespace duplicated values for GENDER and the LANGUAGE dimesnion only has 2 unique values of "en" and "es-419"

### Transaction Data Set

In [9]:
print(transaction_df.head())
print('----------------                         -------------------------')
print(transaction_df.info())

                             RECEIPT_ID PURCHASE_DATE  \
0  0000d256-4041-4a3e-adc4-5623fb6e0c99    2024-08-21   
1  0001455d-7a92-4a7b-a1d2-c747af1c8fd3    2024-07-20   
2  00017e0a-7851-42fb-bfab-0baa96e23586    2024-08-18   
3  000239aa-3478-453d-801e-66a82e39c8af    2024-06-18   
4  00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1    2024-07-04   

                   SCAN_DATE STORE_NAME                   USER_ID  \
0  2024-08-21 14:19:06.539 Z    WALMART  63b73a7f3d310dceeabd4758   
1  2024-07-20 09:50:24.206 Z       ALDI  62c08877baa38d1a1f6c211a   
2  2024-08-19 15:38:56.813 Z    WALMART  60842f207ac8b7729e472020   
3  2024-06-19 11:03:37.468 Z  FOOD LION  63fcd7cea4f8442c3386b589   
4  2024-07-05 15:56:43.549 Z   RANDALLS  6193231ae9b3d75037b0f928   

        BARCODE FINAL_QUANTITY FINAL_SALE  
0  1.530001e+10           1.00             
1           NaN           zero       1.49  
2  7.874223e+10           1.00             
3  7.833997e+11           zero       3.49  
4  4.790050e+10       

- Glad to see there are no null values for the user_id assciated with the purchases, though the lack of Barcodes for a few thousand items could be an issue for summarizing. 

In [10]:
transaction_df.describe(include= "object")

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,FINAL_QUANTITY,FINAL_SALE
count,50000,50000,50000,50000,50000,50000,50000
unique,24440,89,24440,954,17694,87,1435
top,bedac253-2256-461b-96af-267748e6cecf,2024-06-15,2024-09-08 20:00:42.348 Z,WALMART,64e62de5ca929250373e6cf5,1.00,
freq,12,774,12,21326,22,35698,12500


In [11]:
# transaction_df['FINAL_QUANTITY'].min()
# transaction_df['FINAL_SALE'].max()
# sorted(transaction_df['FINAL_SALE'].astype(str).unique())
sorted(transaction_df['FINAL_QUANTITY'].astype(str).unique())

['0.01',
 '0.04',
 '0.09',
 '0.23',
 '0.24',
 '0.28',
 '0.35',
 '0.46',
 '0.48',
 '0.51',
 '0.53',
 '0.62',
 '0.65',
 '0.70',
 '0.83',
 '0.86',
 '0.87',
 '0.94',
 '0.97',
 '0.99',
 '1.00',
 '1.07',
 '1.08',
 '1.13',
 '1.18',
 '1.22',
 '1.23',
 '1.24',
 '1.28',
 '1.34',
 '1.35',
 '1.37',
 '1.40',
 '1.44',
 '1.47',
 '1.50',
 '1.54',
 '1.69',
 '1.80',
 '1.81',
 '1.88',
 '1.89',
 '1.93',
 '1.99',
 '10.00',
 '12.00',
 '16.00',
 '18.00',
 '2.00',
 '2.04',
 '2.10',
 '2.11',
 '2.17',
 '2.18',
 '2.19',
 '2.20',
 '2.25',
 '2.27',
 '2.34',
 '2.39',
 '2.52',
 '2.54',
 '2.57',
 '2.58',
 '2.60',
 '2.61',
 '2.75',
 '2.83',
 '2.89',
 '2.93',
 '276.00',
 '3.00',
 '3.02',
 '3.11',
 '3.24',
 '3.33',
 '3.69',
 '4.00',
 '4.55',
 '5.00',
 '5.53',
 '6.00',
 '6.22',
 '7.00',
 '8.00',
 '9.00',
 'zero']

- A potential issue with the FINAL_QUANTITY is the presence of 'zero' among all of the float (or demical) values, I think I will change those to 0.00 before putting this csv in to a SQLite db
- And with FINAL_SALE the entry of ' ' seems odd, I'm not sure if this is intential or that those should be 0.00 or NaN?

In [12]:
transaction_df[transaction_df['FINAL_SALE']== ' ']

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,
6,000550b2-1480-4c07-950f-ff601f242152,2024-07-06,2024-07-06 19:27:48.586 Z,WALMART,5f850bc9cf9431165f3ac175,4.920091e+10,1.00,
8,000e1d35-15e5-46c6-b6b3-33653ed3d27e,2024-08-13,2024-08-13 18:21:07.931 Z,WALMART,61a6d926f998e47aad33db66,5.200001e+10,1.00,
...,...,...,...,...,...,...,...,...
24990,ffeb1ff4-0db9-4cb9-9574-20ec2db3e5ad,2024-08-25,2024-08-25 12:43:19.456 Z,FOOD CITY,6246d0d989dfe41b042f2d9d,2.484283e+10,1.00,
24992,ffec42ba-c71d-44da-b8d0-eb529632e87a,2024-06-17,2024-06-17 11:17:21.971 Z,WINCO FOODS,6632c5871be3d689ceb875c7,4.119601e+10,1.00,
24994,fff15a3d-25ea-4c36-b84a-91eb4157daf9,2024-07-19,2024-07-23 22:43:04.589 Z,WALMART,6254a7ea3f2afd3c2fd53d22,7.874213e+10,1.00,
24996,fff6c6c8-e002-4262-85ea-25849d9721db,2024-08-30,2024-09-06 18:39:03.161 Z,WALMART,6220b111e9e82c0c6bc24534,7.874237e+10,1.00,


In [13]:
transaction_df[transaction_df['BARCODE'].isnull()]

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
9,0010d87d-1ad2-4e5e-9a25-cec736919d15,2024-08-04,2024-08-04 18:01:47.787 Z,ALDI,66686fc2e04f743a096ea808,NaN,zero,2.29
17,002ee298-d907-40ca-921a-556468571f76,2024-07-15,2024-07-16 16:42:19.211 Z,ALDI,63de64b1dcb50fbd3084f142,NaN,zero,2.49
18,00326689-e763-4b27-9ad5-202fc93609e2,2024-06-19,2024-06-20 08:59:38.397 Z,ALDI,6158642597d737581b5d30ee,NaN,1.00,
60,00a9e033-e49d-45d6-990e-90631f82775e,2024-09-05,2024-09-05 11:10:54.831 Z,ALDI,5d4f08e962fb4a4a58574e7f,NaN,1.00,
...,...,...,...,...,...,...,...,...
49959,45575fc2-6ba3-4913-bdf2-05814e4309e0,2024-06-20,2024-06-20 11:56:29.486 Z,SUPERMERCADOS ECONO,618c25125e388d4f513334b9,NaN,1.00,1.67
49960,7a36db68-c8a7-4b29-b1c2-4cb51ad9f42a,2024-08-29,2024-08-29 14:39:07.868 Z,DOLLAR TREE STORES INC,665e2f887c0469953bfbdb5b,NaN,1.00,1.25
49962,d7cf611f-f07d-4e3d-9a5d-aacc0d56a4a7,2024-07-21,2024-07-21 10:31:52.403 Z,ALDI,61a8f120f6305b3dade12c15,NaN,1.00,1.29
49963,e79c254d-1bf0-4471-8e79-4f52c6b81481,2024-08-24,2024-08-26 15:21:15.492 Z,DOLLAR TREE STORES INC,62f6799b30b23c82198fa01c,NaN,1.00,1.25


- Taking a peek at the 5,762 records in the Transactions data that have a null Barcode to see if these seem like valid entries or any other explanation. From a quick look though it does appear that these items are for legit purchases there is just no item barcode attributed to them. 

  ## Explore Summary
  - With the abscence of a Barcode for some of the items in the PRODUCTS and TRANSACTION csv's there could be a chance that some of the purchased items in the reciepts will not be included in any analysis in a meaningful way. 

## Data Clean up before SQLite DB load
- Change all 'zero' entries in the FINAL_QUANTITY column of Transactions to 0.00

In [14]:
transaction_df['FINAL_QUANTITY'] = transaction_df['FINAL_QUANTITY'].replace('zero', 0.00).astype(float)

# Part II - SQL Queries

In [15]:
#Loading the CSV's into a SQLite database to write the SQL queries for Part II
sql_connect = db.connect('takehome_db')

#Loading each panda df into a table
products_df.to_sql('products',sql_connect, if_exists= 'replace')
user_df.to_sql('users',sql_connect, if_exists= 'replace')
transaction_df.to_sql('transactions',sql_connect, if_exists= 'replace')

#creating a cursor for running queries
c = sql_connect.cursor()

## Closed-Ended Questions

### CEQ #1 What are the top 5 brands by receipts scanned among users 21 and over?

In [24]:
#CE_Q_1
query_1 = '''
         with yr_old_user as (
            select (JulianDay('now') - JulianDay(DATE(BIRTH_DATE)))/365.25 as user_age
            , u.ID
            from users u 
            where user_age >= 21
        )

        select p.BRAND
            , count(t.RECEIPT_ID) as receipts_scanned
        From transactions t
            join yr_old_user yr
               on t.USER_ID = yr.ID
            join products p
                on t.BARCODE = p.BARCODE
                and p.brand is not null --< removing non-branded items so that we can target identifiable brands
        group by p.brand
        order by 2 desc
        limit 5
'''


query_1_df = pd.read_sql_query(query_1, sql_connect)
print(query_1_df)

             BRAND  receipts_scanned
0      NERDS CANDY                 6
1             DOVE                 6
2          TRIDENT                 4
3  SOUR PATCH KIDS                 4
4           MEIJER                 4


### CEQ #2 What are the top 5 brands by sales among users that have had their account for at least six months?

In [23]:
#CE-Q-2
query_2 = '''
        with mt_old_user as (
            select 
                (JulianDay('now') - JulianDay(DATE(CREATED_DATE)))  as account_age_days
                , u.ID
            from users u 
            where account_age_days >= 180 --limiting the user data set to accounts that have only been open for 6 months (180 days) or greater
            )

        select p.BRAND
            , sum(FINAL_SALE) as total_sales
        From transactions t
            join mt_old_user mt
               on t.USER_ID = mt.ID
            join products p
                on t.BARCODE = p.BARCODE
                and p.brand is not null --< removing abscent brands so that we can get a clear understanding.
        group by p.brand
        order by 2 desc
        limit 5 
        
'''


query_2_df = pd.read_sql_query(query_2, sql_connect)
print(query_2_df)

         BRAND  total_sales
0          CVS        72.00
1      TRIDENT        46.72
2         DOVE        42.88
3  COORS LIGHT        34.96
4       QUAKER        16.60


### CEQ #3 What is the percentage of sales in the Health & Wellness category by generation?

In [19]:
#CE-Q-3
query_3 = '''
        with user_gen as (
            select 
                u.ID
                , u.birth_date
                , (strftime('%Y',BIRTH_DATE)*1) as birth_year
                , case when (strftime('%Y',BIRTH_DATE)*1) <= 1927 then 'The Greatest Generation'
                        when (strftime('%Y',BIRTH_DATE)*1) > 1927 and  (strftime('%Y',BIRTH_DATE)*1) <= 1945 then 'The Silent Generation'
                        when (strftime('%Y',BIRTH_DATE)*1) > 1945 and  (strftime('%Y',BIRTH_DATE)*1) <= 1964 then 'Baby Boomers'
                        when (strftime('%Y',BIRTH_DATE)*1) > 1964 and  (strftime('%Y',BIRTH_DATE)*1) <= 1980 then 'Generation X'
                        when (strftime('%Y',BIRTH_DATE)*1) > 1980 and  (strftime('%Y',BIRTH_DATE)*1) <= 1996 then 'Millenials'
                        when (strftime('%Y',BIRTH_DATE)*1) > 1996 and  (strftime('%Y',BIRTH_DATE)*1) <= 2009 then 'Generation Z'
                        when (strftime('%Y',BIRTH_DATE)*1) > 2009 then 'Generation Alpha'
                        end as user_generation
            from users u 
            where true
                and birth_date is not null
            )

        select ug.user_generation
            , sum(t.FINAL_SALE) as total_sales
            , ROUND(SUM(t.FINAL_SALE) * 100.0 / (SELECT SUM(FINAL_SALE) 
                                        FROM transactions t
                                        JOIN user_gen ug 
                                          ON t.USER_ID = ug.ID
                                        JOIN products p 
                                          ON t.BARCODE = p.BARCODE 
                                         AND p.CATEGORY_1 = 'Health & Wellness'), 2) AS percentage_of_total_sales
        From transactions t
            join user_gen ug
               on t.USER_ID = ug.ID
            join products p
                on t.BARCODE = p.BARCODE
                and p.CATEGORY_1 = 'Health & Wellness' 
        group by 1
        order by 2 desc
        
       
'''


query_3_df = pd.read_sql_query(query_3, sql_connect)
print(query_3_df)

  user_generation  total_sales  percentage_of_total_sales
0    Baby Boomers        89.03                      46.94
1      Millenials        59.13                      31.18
2    Generation X        41.50                      21.88


## Open-Ended Questions

### OEQ #1 Who are Fetch’s power users?

In [20]:
#OE-Q-1
query_4 = '''
        
        select
            USER_ID
            , count(distinct RECEIPT_ID) as distinct_receipts
            , sum(FINAL_SALE) as total_dollars_spent
            , count(BARCODE) as items_purchased
            , count(distinct BARCODE) as distinct_items_purchased
        From transactions t
        group by 1
        order by 5 desc
     
       
'''


query_4_df = pd.read_sql_query(query_4, sql_connect)
print(query_4_df)

                        USER_ID  distinct_receipts  total_dollars_spent  \
0      62925c1be942f00613f7365e                 10                49.87   
1      64063c8880552327897186a5                  9                43.72   
2      609af341659cf474018831fb                  7                25.55   
3      6682cbf6465f309038ae1888                  6                37.47   
4      653a0f40909604bae9071473                  4                84.78   
...                         ...                ...                  ...   
17689  599377a8e4b09f6f206e71d5                  1                 0.17   
17690  598b822ae4b0465558c94765                  1                 7.48   
17691  59725916e4b01bd2053089fe                  1                 5.98   
17692  57af9582e4b06f40aeef6f63                  2                15.28   
17693  56242219e4b07364e3e0bef4                  1                 3.18   

       items_purchased  distinct_items_purchased  
0                   20                        10

- Defining Fetch's power users could go in a few directions; the most amount of money spent, the most distinct checkouts registered (Distinct Receipts), the most amount of individual items purchased, the most distinct items purchased. So deciding what defines a power user to Fetch is important and I think it would depend on how Fetch monetizes its user purchase data. If fetch can create more value from using its user's information across many different brands than I think it would be better to focus on users who purchased more variety of items and we could market their information to more brands.

- Which is where I left the last version of the query above to show the top user_ids for users who had made the most distinct list of items

### OEQ #2 Which is the leading brand in the Dips & Salsa category?

In [21]:
#OE-Q-2
query_5 = '''
        
        select
            p.BRAND
            , count(distinct RECEIPT_ID) as distinct_receipts
            , sum(FINAL_QUANTITY) as total_units_purchased
            , sum(FINAL_SALE) as total_dollars_spent
        From transactions t
            join products p
                on t.BARCODE = p.BARCODE
                and p.CATEGORY_2 = 'Dips & Salsa' 
        group by 1
        order by 4 desc
        
     
       
'''


query_5_df = pd.read_sql_query(query_5, sql_connect)
print(query_5_df)

                    BRAND  distinct_receipts  total_units_purchased  \
0                TOSTITOS                 36                   60.0   
1                    None                 21                   35.0   
2              GOOD FOODS                  9                   16.0   
3                    PACE                 24                   38.0   
4              MARKETSIDE                 16                   23.0   
5                  FRITOS                 19                   33.0   
6            HELUVA GOOD!                 15                   20.0   
7    FRESHNESS GUARANTEED                 12                   17.0   
8        DEAN'S DAIRY DIP                 17                   22.0   
9                MARZETTI                 11                   19.0   
10    HIDDEN VALLEY RANCH                  9                   13.0   
11          HIDDEN VALLEY                 10                   22.0   
12                 HERDEZ                  9                   12.0   
13    

- I think it is safe to say that Tostitos is the leading Brand in the Dips & Salsas category, with them leading in the total dollars spent, units purchased, and total amount of distinct purchases.

### OEQ #3 At what percent has Fetch grown year over year?

In [22]:
#OE-Q-3
query_6 = '''
        
        select
           (strftime('%Y',SCAN_DATE)*1) as scan_year
           , sum(FINAL_SALE) as total_dollars_scanned
        From transactions t
        group by 1      

'''


query_6_df = pd.read_sql_query(query_6, sql_connect)
print(query_6_df)

   scan_year  total_dollars_scanned
0       2024               171614.4


- The Transaction data provided is only for the year 2024 and in the months of June through September, so a true Year over Year analysis of Fetch's growth isn't really possible. 

In [25]:
c.close()
sql_connect.close()

# Pasrt III - Stakeholder Communication

- After taking a look at the data provided and doing some initial analysis, there is data enrichment that could help a lot with finding actionable insight:
    1. Building a more robust USER data set that has is able to be matched more with our TRANSACTION data, because to currently only be able to match 262 out of 50,000 transactions (0.5%) with User information is creating a serious gap in analytical ability.

- An interesting insight from analyzing this data though is the large percentage that Baby Boomers make up for the overall sales in the 'Health & Wellness' category. Beating out both younger generations of Gen X and Millenials. This trend is great to see with the older population having good adoption of the Fetch platform and in a category that could be very lucrative. I think further analysis and possible engagement with the Boomer poplutaion could be a great opportunity as they tend to have more disposable income and liesure to shop. 

- Going forward I would like to work with a marketing and/or product leader to get a better definition of what a Power user is for Fetch, as well as work with engineering and data engineering stakeholders to better enrich the USER data to be able to attribute much more to the TRANSACTION data.